In [127]:
import math
import numpy as np
from cvxopt import matrix, glpk
from prettytable import PrettyTable

import sys

np.set_printoptions(threshold=sys.maxsize)


In [128]:
need = [
    {"morning": [8, 8], "evening": [7, 6]},  # monday
    {"morning": [6, 5], "evening": [4, 5]},  # tuesday
    {"morning": [6, 5], "evening": [4, 4]},  # wednesday
    {"morning": [5, 6], "evening": [6, 7]},  # thursday
    {"morning": [7, 8], "evening": [8, 9]},  # friday
    {"morning": [9, 8], "evening": [7, 6]},  # saturday
    {"morning": [6, 5], "evening": [4, 4]},  # sunday
]
daysToWork = 5
regularMonthPayment = 51200
temporaryHourPayment = 380


In [129]:
regularPeopleByDay = [
    [1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
    #
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
]
regularNeedByDay = []
for i in range(len(regularPeopleByDay)):
    regularPeopleByDay[i] = np.concatenate(
        (np.array(regularPeopleByDay[i]),
         np.zeros(len(regularPeopleByDay) * 2))
    )
    regularNeedByDay.append(
        min(need[i]["morning"] if i < len(need)
            else need[i - len(need)]["evening"])
    )
regularPeopleByDay = np.array(regularPeopleByDay) * -1
regularNeedByDay = np.array(regularNeedByDay) * -1

In [130]:
temporaryPeopleByDay = np.array([])
temporaryNeedByDay = []
day = [
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
]
for i in range(len(need)):
    for j in range(len(day)):
        weekDayRow = np.concatenate(
            (
                np.zeros(len(regularPeopleByDay) + i * 4),
                np.array(day[j]),
                np.zeros((len(need) - 1 - i) * 4),
            )
        )
        temporaryPeopleByDay = (
            np.copy(weekDayRow)
            if (i == 0 and j == 0)
            else np.vstack((temporaryPeopleByDay, weekDayRow))
        )
        weekDayNeed = need[i]["morning"] if j < 2 else need[i]["evening"]
        temporaryNeedByDay.append(weekDayNeed[j % 2] - min(weekDayNeed))
temporaryPeopleByDay *= -1 #
temporaryNeedByDay = np.array(temporaryNeedByDay) * -1 #
# temporaryNeedByDay = np.array(temporaryNeedByDay)


In [131]:
# regularAndTemporaryCombined = []
# for i in range(len(regularPeopleByDay)):
    

In [132]:
notNegativeAmount = np.array([])
amountOfX = len(regularPeopleByDay[0])
for i in range(amountOfX):
    row = np.concatenate(
        (np.zeros(i), np.array([-1]), np.zeros(amountOfX - i - 1)))
    notNegativeAmount = (
        np.copy(row) if (i == 0) else np.vstack((notNegativeAmount, row))
    )

In [133]:
temporaryMonthPayment = temporaryHourPayment * 4 * 4
cost = np.concatenate(
    (
        np.full(len(regularPeopleByDay), regularMonthPayment),
        np.full(len(temporaryPeopleByDay), temporaryMonthPayment),
    )
)


In [134]:
targetFunction = matrix(cost, tc="d").T
inequalityLeft = matrix(
    [
        [matrix(regularPeopleByDay).T],
        [matrix(temporaryPeopleByDay).T],
        [matrix(notNegativeAmount).T],
    ],
    tc="d",
).T
inequalityRight = matrix(
    [
        [matrix(regularNeedByDay).T],
        [matrix(temporaryNeedByDay).T],
        [matrix(np.zeros(len(notNegativeAmount))).T],
    ],
    tc="d",
).T

In [135]:
(status, x) = glpk.ilp(
    c=targetFunction,
    G=inequalityLeft,
    h=inequalityRight,
    I=set(range(len(targetFunction))),
)

In [136]:
print(status)
print(x)
for i in range(len(need) - 1):
    for j in range(i * len(need), (i + 1) * len(need)):
        print(x[j], end=" ")
        if j == 13:
            print("")
    print()

optimal
[ 1.00e+00]
[ 2.00e+00]
[ 0.00e+00]
[ 2.00e+00]
[ 2.00e+00]
[ 3.00e+00]
[ 0.00e+00]
[ 2.00e+00]
[ 2.00e+00]
[ 0.00e+00]
[ 4.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 1.00e+00]
[ 0.00e+00]
[ 1.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 1.00e+00]
[ 1.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 1.00e+00]
[ 0.00e+00]
[ 1.00e+00]
[ 0.00e+00]
[ 1.00e+00]
[ 0.00e+00]
[ 1.00e+00]
[ 1.00e+00]
[ 0.00e+00]
[ 1.00e+00]
[ 0.00e+00]
[ 1.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]

1.0 2.0 0.0 2.0 2.0 3.0 0.0 
2.0 2.0 0.0 4.0 0.0 0.0 0.0 

0.0 0.0 1.0 0.0 1.0 0.0 0.0 
1.0 1.0 0.0 0.0 0.0 0.0 1.0 
0.0 1.0 0.0 1.0 0.0 1.0 1.0 
0.0 1.0 0.0 1.0 0.0 0.0 0.0 


In [137]:
weekDays = [
    "Понедельник",
    "Вторник",
    "Среда",
    "Четверг",
    "Пятница",
    "Суббота",
    "Воскресенье",
]


def printResult(resultStatus, resultArray):
    print(f"Статус: {resultStatus}")
    if resultStatus == "optimal":
        regularsTable = PrettyTable(["День недели", "7:00 - 15:00", "15:00 - 23:00"])
        for i in range(len(weekDays)):
            regularsTable.add_row(
                [weekDays[i], resultArray[i], resultArray[i + len(weekDays)]]
            )
        print("График выхода постоянных работников:")
        print(regularsTable)

        temporaryTable = PrettyTable(
            [
                "День недели",
                "7:00 - 11:00",
                "11:00 - 15:00",
                "15:00 - 19:00",
                "19:00 - 23:00",
            ]
        )
        for i in range(len(weekDays)):
            index = len(regularPeopleByDay) + i * len(day)
            temporaryTable.add_row(
                [
                    weekDays[i],
                    resultArray[index],
                    resultArray[index + 1],
                    resultArray[index + 2],
                    resultArray[index + 3],
                ]
            )
        print("График выхода временных работников:")
        print(temporaryTable)

    else:
        print("Оптимальное решение не было найдено.")

In [138]:
printResult(status, x)


Статус: optimal
График выхода постоянных работников:
+-------------+--------------+---------------+
| День недели | 7:00 - 15:00 | 15:00 - 23:00 |
+-------------+--------------+---------------+
| Понедельник |     1.0      |      2.0      |
|   Вторник   |     2.0      |      2.0      |
|    Среда    |     0.0      |      0.0      |
|   Четверг   |     2.0      |      4.0      |
|   Пятница   |     2.0      |      0.0      |
|   Суббота   |     3.0      |      0.0      |
| Воскресенье |     0.0      |      0.0      |
+-------------+--------------+---------------+
График выхода временных работников:
+-------------+--------------+---------------+---------------+---------------+
| День недели | 7:00 - 11:00 | 11:00 - 15:00 | 15:00 - 19:00 | 19:00 - 23:00 |
+-------------+--------------+---------------+---------------+---------------+
| Понедельник |     0.0      |      0.0      |      1.0      |      0.0      |
|   Вторник   |     1.0      |      0.0      |      0.0      |      1.0      |